In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Load the dataset
from google.colab import files
uploaded = files.upload()
data = pd.read_csv('creditcard.csv')

# Step 2: Handle missing values
if data.isnull().sum().sum() > 0:
    data.fillna(data.mean(), inplace=True)

# Step 3: Normalize the data
scaler = StandardScaler()
data['norm_Amount'] = scaler.fit_transform(data[['Amount']])
data['norm_Time'] = scaler.fit_transform(data[['Time']])
data.drop(['Amount', 'Time'], axis=1, inplace=True)

# Independent and dependent variable separation
features = data.drop('Class', axis=1)
labels = data['Class']

# Step 4: Balance the data using SMOTE
print("Applying SMOTE...")
oversampler = SMOTE(sampling_strategy=0.02, random_state=42)
balanced_features, balanced_labels = oversampler.fit_resample(features, labels)

# Step 5: Reduce dimensions using PCA
print("Performing PCA...")
pca_transformer = PCA(n_components=5)
pca_features = pca_transformer.fit_transform(balanced_features)

# Train-test split
train_features, test_features, train_labels, test_labels = train_test_split(
    pca_features, balanced_labels, test_size=0.2, random_state=42
)

# Step 6: Initialize models
model_dict = {
    'Logistic Regression': LogisticRegression(max_iter=500, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=10, min_samples_split=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=50, max_depth=10, max_features='sqrt', random_state=42)
}

from tabulate import tabulate  # Tabular results

# Step 7: Train and Evaluate
eval_results = {}
for model_name, model in model_dict.items():
    print(f"Training {model_name}...")
    model.fit(train_features, train_labels)
    predictions = model.predict(test_features)
    prediction_probabilities = model.predict_proba(test_features)[:, 1]

    eval_results[model_name] = {
        'AUC': roc_auc_score(test_labels, prediction_probabilities),
        'Precision-Recall': precision_recall_curve(test_labels, prediction_probabilities),
        'Report': classification_report(test_labels, predictions, output_dict=True),
        'Confusion Matrix': confusion_matrix(test_labels, predictions),
        'Prediction Probabilities': prediction_probabilities
    }

    print(f"{model_name} AUC: {eval_results[model_name]['AUC']:.4f}")
    print(classification_report(test_labels, predictions))

# Step 8: Visualizations
# ROC Curve
plt.figure(figsize=(10, 6))
for model_name, result in eval_results.items():
    false_positive_rate, true_positive_rate, _ = roc_curve(test_labels, result['Prediction Probabilities'])
    plt.plot(false_positive_rate, true_positive_rate, label=f"{model_name} (AUC={result['AUC']:.4f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.title("ROC Curve for Models")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

# Feature Importance for Random Forest
if 'Random Forest' in model_dict:
    random_forest_model = model_dict['Random Forest']
    random_forest_model.fit(train_features, train_labels)
    rf_importances = random_forest_model.feature_importances_
    plt.figure(figsize=(10, 6))
    sns.barplot(x=rf_importances, y=[f"Principal Component {i+1}" for i in range(len(rf_importances))])
    plt.title("Feature Importance - Random Forest")
    plt.xlabel("Importance")
    plt.ylabel("Features")
    plt.show()

# Confusion Matrix Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(eval_results['Random Forest']['Confusion Matrix'], annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Random Forest')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Correlation Heatmap
plt.figure(figsize=(10, 6))
corr_matrix = pd.DataFrame(balanced_features).corr()
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', cbar=True)
plt.title("Correlation Heatmap After PCA")
plt.show()

# Step 9: Model Comparison
print("\nComparing Models:")
comparison_data = []

for model_name, result in eval_results.items():
    precision = result['Report']['1']['precision']
    recall = result['Report']['1']['recall']
    f1 = result['Report']['1']['f1-score']
    auc = result['AUC']
    comparison_data.append([model_name, precision, recall, f1, auc])

comparison_table = tabulate(
    comparison_data,
    headers=['Model', 'Precision', 'Recall', 'F1-Score', 'AUC'],
    tablefmt='grid'
)
print(comparison_table)

# Identify the best model
best_model_name = max(eval_results, key=lambda x: eval_results[x]['AUC'])
print(f"\nThe best model is: {best_model_name} with an AUC of {eval_results[best_model_name]['AUC']:.4f}")
